### Reading data and dropping unwanted columns

In [111]:
import pandas as pd
df = pd.read_csv('premierleague.csv')
df = df[['Home Team', 'Away Team', 'Winner']]
df

,Home Team,Away Team,Winner
0,MAN UTD,SWANSEA,Away
1,WEST BROM,SUNDERLAND,Draw
2,LEICESTER CITY,EVERTON,Draw
3,WEST HAM,TOTTENHAM,Away
4,QPR,HULL CITY,Away
...,...,...,...
2655,ARSENAL,BRIGHTON,Home
2656,SHEFFIELD UTD,BURNLEY,Home
2657,LEICESTER CITY,TOTTENHAM,Away
2658,WEST HAM,SOUTHAMPTON,Home


### Encoding teams with sklearn lable encoder

In [112]:
from sklearn.preprocessing import LabelEncoder
teamEncoder = LabelEncoder()
teamEncoder.fit(df['Home Team'])

LabelEncoder()

In [113]:
winnerEncoder = LabelEncoder()
winnerEncoder.fit(df['Winner'])

LabelEncoder()

In [114]:
df['Home Team'] = teamEncoder.transform(df['Home Team'])
df['Away Team'] = teamEncoder.transform(df['Away Team'])
df['Winner'] = winnerEncoder.transform(df['Winner'])

### Data preprocessing

In [115]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.preprocessing import StandardScaler

x = df.drop('Winner', axis=1)

sc = StandardScaler()
sc.fit(x)
x = pd.DataFrame(sc.transform(x))

y = df['Winner']
y = to_categorical(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=32)


### ML

In [116]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(8, activation='relu', input_shape=(2, )))
model.add(Dense(6, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 8)                 24        
                                                                 
 dense_28 (Dense)            (None, 6)                 54        
                                                                 
 dense_29 (Dense)            (None, 3)                 21        
                                                                 
Total params: 99
Trainable params: 99
Non-trainable params: 0
_________________________________________________________________


In [117]:
model.fit(x_train, y_train, epochs=10, validation_data=[x_test, y_test], batch_size=40)

Epoch 1/10
54/54 [==============================] - 0s 3ms/step - loss: 1.1371 - accuracy: 0.2726 - val_loss: 1.0996 - val_accuracy: 0.3233
Epoch 2/10
54/54 [==============================] - 0s 1ms/step - loss: 1.0947 - accuracy: 0.3764 - val_loss: 1.0772 - val_accuracy: 0.4248
Epoch 3/10
54/54 [==============================] - 0s 1ms/step - loss: 1.0797 - accuracy: 0.4121 - val_loss: 1.0687 - val_accuracy: 0.4530
Epoch 4/10
54/54 [==============================] - 0s 1ms/step - loss: 1.0737 - accuracy: 0.4319 - val_loss: 1.0641 - val_accuracy: 0.4680
Epoch 5/10
54/54 [==============================] - 0s 1ms/step - loss: 1.0703 - accuracy: 0.4403 - val_loss: 1.0626 - val_accuracy: 0.4680
Epoch 6/10
54/54 [==============================] - 0s 1ms/step - loss: 1.0684 - accuracy: 0.4403 - val_loss: 1.0610 - val_accuracy: 0.4680
Epoch 7/10
54/54 [==============================] - 0s 1ms/step - loss: 1.0669 - accuracy: 0.4403 - val_loss: 1.0601 - val_accuracy: 0.4680
Epoch 8/10
54/54 [==

## Predicting a value

In [119]:
xpred = teamEncoder.transform(['MAN CITY', 'SOUTHAMPTON'])
xpred = sc.transform([xpred])
y = model.predict(xpred)
y = y.argmax(axis=-1)
winnerEncoder.inverse_transform(y)

C:\Users\KarimKohel\miniconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[0.02078614, 0.77524616]])

In [120]:
y = model.predict(xpred)
y = y.argmax(axis=-1)
winnerEncoder.inverse_transform(y)

array(['Home'], dtype=object)